In [35]:
import os
from tqdm import tqdm 
import yfinance as yf
import pandas as pd
import h5py

In [3]:
import pandas as pd
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tickers = pd.read_html(URL)[0]['Symbol'].tolist()

In [85]:
data = yf.download(tickers, start = "2010-01-01", group_by = 'ticker', threads=4)
data  

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted


ISRG                                                   
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2010-01-04   34.133331   34.343334   33.977779   34.307777   34.307777  \
2010-01-05   34.223331   34.435555   33.777779   34.153332   34.153332   
2010-01-06   33.938889   34.754444   33.938889   34.648888   34.648888   
2010-01-07   34.524445   34.888889   34.380001   34.518890   34.518890   
2010-01-08   34.518890   34.744446   34.276669   34.328888   34.328888   
...                ...         ...         ...         ...         ...   
2023-08-17  298.059998  299.570007  289.109985  290.079987  290.079987   
2023-08-18  287.459991  291.059998  284.880005  285.950012  285.950012   
2023-08-21  286.119995  286.880005  282.500000  286.670013  286.670013   
2023-08-22  287.730011  289.079987  285.730011  287.160004  287.160004   
2023-08-23  287.559998  292.690002  287.559998  290.390015  290.390015   

                           GEN                                   ...   
             Volume       Open       High        Low      Close  ...   
Date                                                             ...   
2010-01-04  3033900  18.040001  18.530001  18.010000  18.400000  ...  \
2010-01-05  3690900  18.400000  18.440001  18.139999  18.299999  ...   
2010-01-06  4995900  18.219999  18.459999  18.139999  18.360001  ...   
2010-01-07  2651400  18.469999  18.469999  18.129999  18.320000  ...   
2010-01-08  2657700  18.250000  18.500000  18.209999  18.370001  ...   
...             ...        ...        ...        ...        ...  ...   
2023-08-17  2061400  20.540001  20.680000  20.150000  20.160000  ...   
2023-08-18  2983400  19.750000  20.469999  19.670000  20.370001  ...   
2023-08-21  1776700  20.410000  20.860001  20.360001  20.790001  ...   
2023-08-22  1161400  20.900000  21.090000  20.510000  20.530001  ...   
2023-08-23  1282016  20.540001  20.790001  20.455000  20.700001  ...   

                  WBA                                        PNW              
                  Low      Close  Adj Close    Volume       Open       High   
Date                                                                          
2010-01-04  36.639999  37.299999  25.140753   8171000  36.660000  37.090000  \
2010-01-05  36.619999  37.000000  24.938557  10843200  36.770000  37.020000   
2010-01-06  35.919998  36.720001  24.749838   8945400  36.340000  36.759998   
2010-01-07  36.470001  36.939999  24.898111   5040700  36.590000  36.709999   
2010-01-08  36.650002  36.990002  24.931808   6139800  36.540001  36.939999   
...               ...        ...        ...       ...        ...        ...   
2023-08-17  27.120001  27.160000  26.679998  13469000  77.320000  78.129997   
2023-08-18  26.510000  26.540001  26.540001   8928900  77.449997  78.650002   
2023-08-21  26.209999  26.440001  26.440001   6833100  78.190002  78.190002   
2023-08-22  26.120001  26.540001  26.540001   9178800  78.589996  79.059998   
2023-08-23  26.275000  26.540001  26.540001   5883741  79.349998  79.489998   

                                                      
                  Low      Close  Adj Close   Volume  
Date                                                  
2010-01-04  36.590000  36.880001  21.100122   873500  
2010-01-05  36.110001  36.380001  20.814056  1432200  
2010-01-06  36.189999  36.759998  21.031467   961100  
2010-01-07  36.380001  36.689999  20.991419   686000  
2010-01-08  36.279999  36.919998  21.123005   614000  
...               ...        ...        ...      ...  
2023-08-17  77.209999  77.330002  77.330002   813300  
2023-08-18  77.449997  78.250000  78.250000  1062600  
2023-08-21  76.680000  77.680000  77.680000   950300  
2023-08-22  78.099998  78.699997  78.699997  1429800  
2023-08-23  78.209999  78.900002  78.900002   849231  

[3433 rows x 3018 columns]

In [98]:
tickers = data.columns.get_level_values(0).unique()
# Create a dictionary to store separate DataFrames
dfs_by_ticker = {}

# Split the DataFrame based on tickers
for ticker in tickers:
    sub_df = data.xs(key=ticker, level=0, axis=1)
    dfs_by_ticker[ticker] = sub_df

In [105]:
path = r"C:\Users\natha\OneDrive\Desktop\C++\Nexus\AgisCoreTest\data\SPY_DAILY"
h5_path = _path = os.path.join(path, "data.h5")

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    df_mid = dfs_by_ticker[ticker].copy()
    df_mid['50_MA'] = df_mid['Close'].rolling(window=50).mean()
    df_mid['200_MA'] = df_mid['Close'].rolling(window=200).mean()
    df_mid["SIGNAL"] = (df_mid["50_MA"] > df_mid["200_MA"]).astype(int)
    df_mid.dropna(inplace=True)
    cols = df_mid.columns
    data = df_mid[cols].to_numpy()
    if len(df_mid) == 0: continue
    with h5py.File(_path, "a") as file:
        # Create a new dataset and save the data
        file.create_dataset(
                f"{ticker}/datetime",
                data=df_mid.index.map(lambda timestamp: int(timestamp.timestamp() * 1e9)).to_numpy()
            )
        dataset = file.create_dataset(f"{ticker}/data", data=data)

        # Store column names as attributes
        for col_name in cols:
            dataset.attrs[col_name] = col_name



100%|██████████| 503/503 [00:03<00:00, 140.11it/s]


In [107]:
sum([len(df) for df in dfs_by_ticker.values()])

1726799

In [77]:
i = 0
df = dfs[i].copy()
ticker = tickers[i]
df.columns = [col[col.find(" ")+1:] for col in df.columns]
df['50_MA'] = df['Close'].rolling(window=50).mean()
df['200_MA'] = df['Close'].rolling(window=200).mean()
df["SIGNAL"] = (df["50_MA"] > df["200_MA"]).astype(int)
df.dropna(inplace=True)
df

,Open,High,Low,Close,Adj Close,Volume,50_MA,200_MA,SIGNAL
Date,,,,,,,,,
2010-10-18,89.089996,89.919998,88.690002,89.660004,60.579533,2692000,85.001800,82.96620,1
2010-10-19,88.750000,88.959999,87.510002,88.550003,59.829544,3888100,85.012200,82.99385,1
2010-10-20,88.730003,89.949997,88.599998,89.470001,60.451138,2819400,85.059400,83.02870,1
2010-10-21,89.870003,90.900002,89.720001,90.550003,61.180870,5633800,85.188401,83.06310,1
2010-10-22,90.449997,90.629997,90.019997,90.440002,61.106529,2310800,85.317001,83.09665,1
...,...,...,...,...,...,...,...,...,...
2023-08-17,101.230003,102.720001,101.099998,101.800003,100.300003,3404100,103.112200,110.57230,0
2023-08-18,99.980003,100.489998,99.580002,100.010002,100.010002,2632100,103.092401,110.44340,0
2023-08-21,100.000000,100.099998,98.529999,99.190002,99.190002,2302700,103.061001,110.31285,0


In [74]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '50_MA',
       '200_MA', 'SIGNAL'],
      dtype='object')